## HW1.0.0. 

**Define big data. Provide an example of a big data problem in your domain of expertise.** 

Big Data is a term used to describe a massive volume of diverse data, both structured and unstructured, that is so large and 
fast-moving that it’s difficult or impossible to process using traditional databases and software technology. In most enterprise 
scenarios, the data is too enormous, streaming by too quickly at unpredictable and variable speeds, and exceeds current 
processing capacity.

**What is a race condition in the context of parallel computation? Give an example.**


**What is MapReduce?**

MapReduce is a processing technique and a program model for distributed computing based on java. The MapReduce algorithm 
contains two important tasks, namely Map and Reduce. Map takes a set of data and converts it into another set of data, where 
individual elements are broken down into tuples (key/value pairs). Secondly, reduce task, which takes the output from a map 
as an input and combines those data tuples into a smaller set of tuples. As the sequence of the name MapReduce implies, the 
reduce task is always performed after the map job.

**How does it differ from Hadoop?**

First off, Map Reduce is an algorithm, Hadoop is an Ecosystem. Both are indepent of underlying hardware or programming 
language or technology stack. Now, Hadoop leverages this algorithm to split up tasks accross multiple processing nodes and 
then recombining them. So, you can say the Apache hadoop ecosystem leverages the Map reduce algorithm over a Java platform 
to do parallel, distibuted multiprocessing.



In [ ]:
## HW1.0.1: 

Here is an example of functional programming in basic python in terms of mappers and reducers (by way of example):

**#EXAMPLE Mapper functions in Python:**

In [ ]:
def fahrenheit(T):
    return ((float(9)/5)*T + 32)

def celsius(T):
    return (float(5)/9)*(T-32)

temperatures = (36.5, 37, 37.5, 38, 39)
F = map(fahrenheit, temperatures)
#returns  97.7  98.6  99.5 100.4 102.2
C = map(celsius, F)

#EXAMPLE Reducer function in Python
import functools
functools.reduce(lambda x,y: x+y, [47,11,42,13])
#returns 113

import functools as reduce
print "Average temp is %fF" % (reduce(lambda x,y: x+y, F)/len(F) )
#returns Average temp is 99.68F

In [ ]:
**Which programming paradigm is Hadoop based on? Explain and give a simple example of functional programming in raw python code 
and show the code running. E.g., in raw python find the average length of a string in and of strings using a python "map-reduce" 
(functional programming) job (similar in style to the above). Alternatively, you can do this in python Hadoop Streaming.**

In [ ]:
strings = ["str1", "string2", "w261", "MAchine learning at SCALE"]
.......

import functools as reduce
temperatures = (36.5, 37, 37.5, 38, 39)
F = map(fahrenheit, temperatures)
print "Average temp is %fF" % (reduce(lambda x,y: x+y, F)/len(F) )
#returns Average temp is 99.68F

map(sqr, items)

In [ ]:
# MapReduce example - avg length of strings
import functools

def length_string(S):
    return(len(S))

strings = ["str1", "string2", "w261", "MAchine learning at SCALE"]

A = map(length_string, strings)

print('Average (Mean) Length for the input strings is %i' % (functools.reduce(lambda x, y: x + y, A) / len(A)))

# print('Average (Mean) Length for the input strings is {:f}'.format(functools.reduce(lambda x, y: x + y, A) / len(A)))

## HW1.1 Cross fold validation (*)

## HW1.2: WORDCOUNT

Using the Enron dataset and Hadoop MapReduce streaming (or MRJob), write the mapper/reducer job that will determine the word count (number of occurrences) of each white-space delimitted token (assume spaces, fullstops, comma as delimiters). Examine the word “assistance” and report its word count results.
CROSSCHECK: >grep assistance enronemail_1h.txt|cut -d$'\t' -f4| grep assistance|wc -l
8
   #NOTE  "assistance" occurs on 8 lines but how many times does the token occur? 10 times! This is the number we are looking for!

In [ ]:

%%writefile wordcount.py   
# Note that this code works even when reducer is not set to 1
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from mrjob.job import MRJob
import re, string

class MRJobWordCount(MRJob):

    def mapper(self, _, line):
        # Creates regular expression to modify punctuation characters so that they will not mess up regex
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        # Splits the lines by the first 2 tabs. Keeps everything after the second tab. 
        token = line.strip().split('\t', 2)[-1]
        # Uses the regex above to replace puctuation with a ' ' and puts all words in lower case
        token = regex.sub(' ', token.lower())
        # Converts all white space that is not a ' ' into a ' ', including duplicate spaces
        token = re.sub( '\s+', ' ', token )
        
        # Creates a list of the words
        words = token.split()

        # for loop that results in only words greater than length 1 being yielded
        for word in words:
            if len(word) > 1:
                yield (word, 1)
   
    # Creats sum by word for each mapper - like a per-mapper reducer
    def combiner(self, word, counts):
        yield (word, sum(counts))

    # Creates an empty dictionary called results, then sums 
    def reducer(self, word, counts):
        yield (word, sum(counts))
    
if __name__ == '__main__':
    MRJobWordCount.run()

In [ ]:
!python wordcount.py enronemail_1h.txt

In [ ]:
# Check the number of times assistance appears 
!grep -o assistance enronemail_1h.txt | wc -l

In [ ]:
# Output word counts to txt for later use

%reload_ext autoreload
%autoreload 2

from numpy import random
from wordcount import MRJobWordCount 
     
count_data = 'enronemail_1h.txt'

mr_job = MRJobWordCount(args=[count_data])
model_stats = {}
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access to the output reducer/reducer_final of 
    # the last step in MRJobWordCount
    for line in runner.stream_output():
        key, value =  mr_job.parse_output_line(line)
        print key, value
        model_stats[key] = value            
    # Store model locally
    with open('model1.txt', 'w') as f:
        for k in model_stats.keys():
            f.writelines( k + "\t"+ str(model_stats[k]) +"\n")
print model_stats

In [ ]:
model_stats = {}
records = [s.split('\n')[0].split('\t') for s in open("model1.txt").readlines()]
for word, count in records:
    model_stats[word] =  map(int, count.split(","))
model_stats

In [ ]:
## HW1.2.1

Using Hadoop MapReduce (or MRJob) and your wordcount job (from HW1.2) determine the top-10 occurring tokens 
(most frequent tokens) using a single reducer.

In [ ]:
from collections import Counter

# Using model_stats created in MRJob above to get 10 most common words
Counter(model_stats).most_common(10)

In [ ]:
## HW1.3: Multinomial NAIVE BAYES with NO Smoothing using a single reducer

Using the Enron data from HW1 and Hadoop MapReduce (or MRJob), write a mapper/reducer job(s) that will both learn Naive Bayes 
classifier and classify the Enron email messages using the learnt Naive Bayes classifier. Use all white-space delimitted tokens 
as independent input variables (assume spaces, fullstops, commas as delimiters). Note: for multinomial Naive Bayes, 
the Pr(X=“assistance”|Y=SPAM) is calculated as follows:

    the number of times “assistance” occurs in SPAM labeled documents / the number of words in documents labeled SPAM

    E.g., “assistance” occurs 5 times in all of the documents Labeled SPAM, and the length in terms of the number of words in 
all documents labeled as SPAM (when concatenated) is 1,000. Then Pr(X=“assistance”|Y=SPAM) = 5/1000. Note this is a multinomial 
estimation of the class conditional for a Naive Bayes Classifier. No smoothing is needed in this HW. Multiplying lots of 
probabilities, which are between 0 and 1, can result in floating-point underflow. Since log(xy) = log(x) + log(y), it is better 
to perform all computations by summing logs of probabilities rather than multiplying probabilities. Please pay attention to 
probabilites that are zero! They will need special attention. Count up how many times you need to process a zero probabilty 
for each class and report.

    Report the performance of your learnt classifier in terms of misclassifcation error rate of your multinomial Naive Bayes 
    Classifier. Plot a histogram of the posterior probabilities (i.e., Pr(Class|Doc)) for each class over the training set. 
    Summarize what you see.

    Error Rate = misclassification rate with respect to a provided set (say training set in this case). It is more formally 
    defined here:

Let DF represent the evalution set in the following: Err(Model, DF) = |{(X, c(X)) ∈ DF : c(X) != Model(x)}| / |DF|

Where || denotes set cardinality; c(X) denotes the class of the tuple X in DF; and Model(X) denotes the class inferred by 
the Model “Model”

NOTE: please assume one reducer.

In [ ]:
# Frist splitting data into 70% for training, 15% for valdiation, and 15% for testing 
# Works even if number of lines != 100
import random

with open("enronemail_1h.txt", "r") as f:
    data = f.read().split('\n')

# Randomly sort data to avoid any bias from the order in original data
random.shuffle(data)

num_lines = len(data)
num_lines_70pct = int(.7*num_lines)
num_lines_85pct = int(.85*num_lines)


train_data = data[:num_lines_70pct]
validation_data = data[num_lines_70pct:num_lines_85pct]
testing_data = data[num_lines_85pct:]

with open("enron_training_data.txt", "w") as f:
    f.write("\n".join(train_data))
with open("enron_validation_data.txt", "w") as f:
    f.write("\n".join(validation_data))
with open("enron_testing_data.txt", "w") as f:
    f.write("\n".join(testing_data))

In [ ]:
%%writefile naive_bayes_enron.py

# NOTE: I could set this to use one reducer as was asked, but there is no need to with the code below. 

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
 
from collections import defaultdict
 
from mrjob.job import MRJob
from mrjob.job import MRStep

import re, string

# Stores the number of emails by classifier
email_counts = dict()
 
# Stores the number of words by classifier
word_counts = dict()

class NaiveBayesEnron(MRJob):
    """
    A MRJob class for a Naive Bayes probability domain calculation of whether an email
    is SPAM or HAM (not-spam).
    """
 
    def __init__(self, *args, **kwargs):
        super(NaiveBayesEnron, self).__init__(*args, **kwargs)
     
    def configure_options(self):
        """
        Adding this allows switching between smoothing methods w/o having to copy all code 
        for each method and for changing the min length of words included in the model
        """
        super(NaiveBayesEnron, self).configure_options()
        self.add_passthrough_option(
            '--smoothing', default='none', choices=['none', 'laplace', 'jm'], 
            help='Specifies the type of smoothing to use'
        )
    
        self.add_passthrough_option(
            '--min-word-frequency', default=1, dest='min_word_frequency', type='int',
            help='Specifies the min length of a word to be included in model'
        )
        
        self.add_passthrough_option(
            '--lambda', default=0.3, dest='jm_lambda', type='float',
            help='Specifies the desired lambda value for JM smoothing'
        )
        
    def steps(self):
        out = [
            MRStep(
                mapper = self.mapper_one,
                combiner = self.combiner_one,
                reducer = self.reducer_one
            )
        ]
        
        if self.options.smoothing == 'laplace': 
            out.append(MRStep(
                reducer = self.reducer_laplace
            ))
        
        elif self.options.smoothing == 'jm':
            out.append(MRStep(
                reducer = self.reducer_jm
            ))
            
        else:
            out.append(MRStep(
                reducer = self.reducer_unsmoothed
            ))
        
        return out
 
    def mapper_one(self, _, email):
        """
        Read each email and map it to a collection of words and
        counts
 
        :param _:
            There is no key here because we're loading raw text
        :param email:
            An email to be processed
        :return:
            Yields a generator that returns (key, value) tuples for each unique
            word present in the email.
                * key: a tuple containing the word and the classifier for
                    the email
                * value: the number of times that word appeared in the email
        """
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        _, classifier, token = email.strip().split('\t', 2)
        token = regex.sub(' ', token.lower())
        token = re.sub( '\s+', ' ', token )
        
        words = token.split()
 
        # Counter for the number of emails processed by classifier
        yield (('EMAILS', classifier), 1)
 
        for word in set(words):                
            count = words.count(word)
 
            yield ((word, classifier), count)
 
            # Counter for the number of words processed by classifier
            yield (('WORDS', classifier), count)
 
 
    def combiner_one(self, word_classifier, counts):
        """
        Combine the raw output of each mapper so that the value for each key
        is now the number of times each word-classifier appeared in that
        mapper (not just in a particular email). This reduces the amount of
        data each mapper sends to the reducer.
 
        :param word_classifier:
            A tuple with a word and classifier combination
        :param counts:
            The number of counts that word + classifier combination was found
            in each email processed by a particular mapper.
        :return:
            Yields a generator that returns (key, value) tuples for each unique
            word processed by a particular mapper.
                * key: a tuple containing the word and the classifier for
                    the email
                * value: the number of times that word + classifier
                    combination appeared in the mapper
        """
 
        yield (word_classifier, sum(counts))
 
    def reducer_one(self, word_classifier, counts):
        """
        Collects the results of each mapper to get the total counts for each
        word + classifier combination within the processed document.
 
        :param word_classifier:
            A tuple with a word and classifier combination
        :param counts:
            The number of counts that word + classifier combination was found
            in each email processed by a particular mapper.
        :return:
            Yields a generator that returns (key, value) tuples for each unique
            word processed by a particular mapper.
                * key: a tuple containing the word and the classifier for
                    the email
                * value: the number of times that word + classifier
                    combination appeared in the mapper
        """
        # Calculate the count for each key 
        # e.g., for key (word, classifier) sum the list coming in from the combiner/mapper 
        # Could look like this: (the, 0):[1,4,1] will turn into (the, 0):6
        total_count = sum(counts)
        # split up the word_classifier tuple into word, classifier
        word, classifier = word_classifier
 
        # store the total number of words for a paticular classifier
        # By time the reducer is done, this will add the word count for each classifier to the word_counts dict
        # adding the return at the end of this it discards the "WORDS" keys 
        # as they are no longer needed after getting total_count
        if word == 'WORDS':
            if classifier not in word_counts:
                word_counts[classifier] = 0
                
            word_counts[classifier] += total_count
            return
        
        # store the total number of emails for a paticular classifier
        # By time the reducer is done, this will add the number of emails for each classifier to the email_counts dict 
        # Changed word from 'EMAILS' to 'PRIOR' because how we calculate priors is with the total num emails per classifier
        if word == 'EMAILS':
            email_counts[classifier] = total_count
            word = 'PRIOR'
 
        # This will be reduced again as we may get outputs for the same key more than once (if word appeared in both classifiers)
        # for example, (the, {0:3}) and (the, {1:5})
        # What the next reducer will receive in this case will look like this: (the, [{0:3}, {1:5}])
        if total_count < self.options.min_word_frequency:
            # Removes words from the total count if they were below the threshold and doesn't yield them to the
            # second stage reducers
            if classifier not in word_counts:
                word_counts[classifier] = 0
            word_counts[classifier] -= total_count
        else:
            yield (word, {classifier: total_count})
 
    def reducer_unsmoothed(self, word, classified_counts):
        """
        After the first reducer has completed, which populates the global
        variables needed to calculate probabilities from word counts, the
        results stream is processed to return the smoothed probabilities
 
        :param word:
            A unique word that was processed within the document
        :param classified_counts:
            A collection of dictionaries where the keys in each dictionary
            are the classifiers and the values are the number of times that
            word appeared within that classifier.
        :return:
            Prints the final Naive Bayes probabilities for later use
        """
 
        # dictionary where if the key doesn't exist, just give me 0
        combined = defaultdict(lambda: 0)
    
        # Creatse a single dictionary for a given word, with the key = classifier, and value = count
        # Using the example above this will result in {0:3, 1:5}
        for entry in classified_counts:
            for classifier, count in entry.items():
                combined[classifier] += count
 
        # for each classifier calculate the probability that the given word occured
        # if the word is 'PRIOR' it calculates the prior probability for that classifier
        for classifier in email_counts.keys():
            count = combined.get(classifier, 0)
 
            if word == 'PRIOR':
                probability = count / sum(email_counts.values())
            else:
                probability = count / word_counts[classifier]
 
            yield (word, classifier), probability
    
    def reducer_laplace(self, word, classified_counts):
        """
        This is the laplace smoothed version of the second stage reducer (reducer_unsmoothed)
        """
        
        combined = defaultdict(lambda: 0)
        
        for entry in classified_counts:
            for classifier, count in entry.items():
                combined[classifier] += count
 
        for classifier in email_counts.keys():
            count = combined.get(classifier, 0)
 
            if word == 'PRIOR':
                probability = count / sum(email_counts.values())
            else:
                probability = (count + 1) / (word_counts[classifier] + 2)
 
            yield (word, classifier), probability
    
    def reducer_jm(self, word, classified_counts):
        """
        This is the Jelinek-Mercer smoothed version of the second stage reducer (reducer_unsmoothed)
        """
        
        combined = defaultdict(lambda: 0)
        
        for entry in classified_counts:
            for classifier, count in entry.items():
                combined[classifier] += count
 
        for classifier in email_counts.keys():
            count = combined.get(classifier, 0)

        for classifier in email_counts.keys():
            count = combined.get(classifier, 0)
            jm_lambda = self.options.jm_lambda
        
            if word == 'PRIOR':
                probability = count / sum(email_counts.values())
            else:
                total_this_word = sum(combined.values())
                total_all_words = sum(word_counts.values())
                
                probability = (
                    (1 - jm_lambda) * (count / word_counts[classifier]) +
                    (jm_lambda * total_this_word / total_all_words)
                )
                
            yield (word, classifier), probability 

if __name__ == '__main__':
    NaiveBayesEnron.run()

In [ ]:
# Build model code
%reload_ext autoreload
%autoreload 2

import naive_bayes_enron as nbe 


def build_model(training_filename, output_filename, smoothing_type='none', min_word_frequency=1, jm_lambda=0.3):
    nbe.word_counts = dict()
    nbe.email_counts = dict()
    mr_job = nbe.NaiveBayesEnron(
        args=[
            training_filename,
            '--smoothing={}'.format(smoothing_type),
            '--min-word-frequency={}'.format(min_word_frequency),
            '--lambda={}'.format(jm_lambda)
        ]
    )
    model_stats = dict()
    
    with mr_job.make_runner() as runner: 
        runner.run()
        # stream_output: get access to the output reducer/reducer_final of 
        # the last step in NaiveBayesEnron
        for line in runner.stream_output():
            key, value =  mr_job.parse_output_line(line)
            word = key[0]
            classifier = int(key[1])

            if word not in model_stats:
                probabilities = ['0', '0']
                probabilities[classifier] = str(value)
                model_stats[word] = probabilities                        
            else:
                model_stats[word][classifier] = str(value)

        # Store model locally
        with open(output_filename, 'w') as f:
            for word, probabilities in model_stats.items():
                f.writelines(word + "\t" + "\t".join(probabilities) + "\n")

In [ ]:
# build unsmoothed model    
build_model(    
    training_filename='enron_training_data.txt',
    smoothing_type='none',
    output_filename='enron_model_unsmoothed.txt',
    min_word_frequency=1
)

In [ ]:
%%writefile naive_bayes_enron_classifier.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
from mrjob.job import MRJob
from mrjob.step import MRStep
import os, re, string, math

counts = []
email_posteriors = dict()

class NaiveBayesEnronClassifier(MRJob):


    def __init__(self, *args, **kwargs):
        super(NaiveBayesEnronClassifier, self).__init__(*args, **kwargs)
        
    def steps(self):
        return [
            MRStep(
                mapper_init=self.mapper_init, 
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer  
            ),
            MRStep(
                reducer=self.reducer_two
            )
        ]

    def configure_options(self):
        super(NaiveBayesEnronClassifier, self).configure_options()
        
        self.add_file_option('--model')
        
    def mapper_init(self): 
        self.model_stats = {}

        with open(self.options.model, "r") as f:
            lines = f.read().split('\n')
        
        split_lines = [line.split('\t') for line in lines]
        del lines # Deleting the the line variable to save memory
        
        for entry in split_lines:
            word = entry[0]
            probabilities = [float(p) for p in entry[1:]]
            self.model_stats[word] = probabilities
    
    def mapper(self, _, line):
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        _, classifier, token = line.strip().split('\t', 2)
        token = regex.sub(' ', token.lower())
        token = re.sub( '\s+', ' ', token )
        
        # set initial values to the priors obtainined in model_stats
        posteriors = [math.log10(prior) for prior in self.model_stats['PRIOR']]
        
        for word in token.split():
            # Get the value for the word, or if it is not in model_stats, return 0 as the default
            probabilities = self.model_stats.get(word, [0, 0]) 
            
            # converts all 0s to 1s - note that log10 of 1 = 0
            probabilities = [p if p > 0 else 1 for p in probabilities] 
           
            for index, probability in enumerate(probabilities):
                if probability == 1:
                    yield('zero_count_{}'.format(index), 1)
        
            posteriors = [x + math.log10(y) for x, y in zip(posteriors, probabilities)]
            
        for index, posterior in enumerate(posteriors):
            yield('post_{}'.format(index), posterior)
            
        # Determine which class the email belongs to (SPAM vs HAM)
        # If p_class_0 = p_class_1, class is unknown
        max_posterior = max(posteriors)
        max_counts = [1 if p == max_posterior else 0 for p in posteriors]
        if sum(max_counts) > 1:
            predicted_class = -1
        else:
            predicted_class = posteriors.index(max_posterior)
            
        # Determine if our predicted class is correct
        if predicted_class == int(classifier):
            key = 'correct'
        else:
            key = 'incorrect'
            
        yield (key, 1)
        yield ('email_count', 1)

    def combiner(self, key, values):
        
        if key.startswith('post_'):
            # Keep a list of the posteriors for each class type for histogram
            key = key.split('_')[-1]
            if key not in email_posteriors:
                email_posteriors[key] = []
            email_posteriors[key] += list(values)
            return
        
        yield (key, sum(values))
        
    def reducer(self, key, values):
        
        if key.startswith('post_'):
            # Keep a list of the posteriors for each class type for histogram
            key = key.split('_')[-1]
            if key not in email_posteriors:
                email_posteriors[key] = []
            email_posteriors[key] += list(values)
            return
        
        count = sum(values)
        
        if key in ['correct', 'incorrect']:
            counts.append(count)
            
        yield (key, count)
      
    def reducer_two(self, key, values):
        values = list(values)
        
        if key in ['correct', 'incorrect']:
            rate = sum(values) / sum(counts)
            out_key = 'error_rate' if key == 'incorrect' else 'accuracy'
            yield (out_key, rate)
        else:
            yield (key, sum(values))

if __name__ == '__main__':
    NaiveBayesEnronClassifier.run()

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import naive_bayes_enron_classifier as nbec


def run_classifier(trial_name, testing_filename, model_filename):
    model_path = os.path.join(
        os.path.abspath(os.path.curdir), 
        model_filename
    )
    nbec.counts = []
    nbec.email_posteriors = dict()
    mr_job = nbec.NaiveBayesEnronClassifier(
        args=[
            testing_filename,
            '--model={}'.format(model_path)
        ]
    )
    out = {'trial_name': trial_name, 'error_rate': 0, 'accuracy': 0, 'posteriors': nbec.email_posteriors}
    
    with mr_job.make_runner() as runner: 
        runner.run()
        for line in runner.stream_output():
            key, value =  mr_job.parse_output_line(line)
            out[key] = value
                
    return out

In [ ]:
results = run_classifier('unsmoothed_test', 'enron_testing_data.txt', 'enron_model_unsmoothed.txt')
print('\n'.join(['* {}: {}'.format(k, v) for k, v in results.items()]))

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

# Count up how many times you need to process a zero probabilty for each class and report.

print('Zero Probability Counts:')
print('------------------------')
for key, value in results.items():
    if key.startswith('zero_count_'):
        print('Classifier[{}] = {}'.format(
                key.rsplit('_', 1)[-1],
                value
        ))

# Note that I split my data into training/test/validation from the beginning, so my numbers might not match yours
# I did this because if you use the full data set for training and testing, it will falsely inflate your accuracy
# Providing counts from my 15% testing data

for classifier, posteriors in results['posteriors'].items(): 
    n, bins, patches = plt.hist(posteriors, 20, normed=1)

    plt.xlabel('Probability')
    plt.ylabel('Frequency')
    plt.title('{} Classifier Posterior Probabilities'.format(classifier))
    plt.grid(True)
    plt.show()

In [ ]:
## HW1.4: Multinomial Naive Bayes with Smoothing

**HW1.4.0: Repeat HW1.3 with the following modification: use Laplace plus-one smoothing. Compare the misclassifcation error rates 
for HW1.3 versus HW1.4 and explain the differences.**

In [ ]:
build_model(    
    training_filename='enron_training_data.txt',
    smoothing_type='laplace',
    output_filename='enron_model_laplace.txt',
    min_word_frequency=1
)

In [ ]:
run_classifier('laplace_test', 'enron_testing_data.txt', 'enron_model_laplace.txt')

In [ ]:
## HW 1.4.1 Jelinek-Mercer (JM) smoothing*

With different smoothing methods, p(wk|ci) (i.e., the word class conditionals) will be computed differently. We consider 
Jelinek-Mercer (JM) smoothing as an alternative to Laplace Let c(w, ci) denote the frequency of word w in category ci, p(w|C) 
be the maximum likelihood estimation of word w in collection C (relative frequency) and let |C for classi| denote the length 
of the classi. Then:
    
1) Jelinek-Mercer (JM) smoothing:

λp(w|ci) = (1 − λ) * c(w, ci)/sum_over_wJ_in_V(c(wJ, ci)) + λ p(w|C)

Where c(w, ci)/sum_over_wJ_in_V(c(wJ, ci)) essential denotes the relative frequency of word w in class ci, i.e., Pr(w|ci) and 
one can set λ = 0.3 by default.

In [ ]:
build_model(    
    training_filename='enron_training_data.txt',
    smoothing_type='jm',
    output_filename='enron_model_jm.txt',
    min_word_frequency=1,
    jm_lambda=0.3
)

In [ ]:
run_classifier('jm_test', 'enron_testing_data.txt', 'enron_model_jm.txt')

In [ ]:
## HW1.4.2 Split data in to training, validation and testing data subsets

Split the data using MRJob into three subsets in the following proportions (70% for training, 15% for valdiation, 
and 15% for testing). Train Multinomial Naive Bayes classifiers using Laplace plus-one smoothing and using Jelinek-Mercer (JM)
smoothing where you consider different hyperparameter values for λ. Please consider λ in {0.0, 0.1, 0.3, 0.5, 0.7, 1}. Present 
a table compare the results of the different approaches: each row is the approach taken (e.g., Multinomial Naive Bayes with 
Laplace+1, or Multinomial Naive Bayes with with JM= 0.3 for λ =0.3) and a column for error rate on the training, validation and 
test data sets. Present a graph also (in python) consisting of three curves (where the x-axis represents the approach taken and 
the y-axis represents the error rate).

In [ ]:
import pandas as pd

testing_filename = 'enron_testing_data.txt'
validation_filename = 'enron_validation_data.txt'
model_filename = 'enron_model_trial.txt'
trials = []

def run_trial(trial_name, smoothing_type, jm_lambda=0.3, min_word_frequency=1):
    
    build_model(    
        training_filename='enron_training_data.txt',
        smoothing_type=smoothing_type,
        output_filename=model_filename,
        min_word_frequency=min_word_frequency,
        jm_lambda=jm_lambda
    )
    
    out = {'name': trial_name}
    
    results = run_classifier(trial_name, testing_filename, model_filename)
    out['error_test'] = results['error_rate']
    
    results = run_classifier(trial_name, validation_filename, model_filename)
    out['error_validate'] = results['error_rate']
    
    return out

trials.append(run_trial('unsmoothed', 'none'))
trials.append(run_trial('laplace', 'laplace'))
trials.append(run_trial('jm = 0.0', 'jm', jm_lambda=0.0))
trials.append(run_trial('jm = 0.1', 'jm', jm_lambda=0.1))
trials.append(run_trial('jm = 0.3', 'jm', jm_lambda=0.3))
trials.append(run_trial('jm = 0.5', 'jm', jm_lambda=0.5))
trials.append(run_trial('jm = 0.7', 'jm', jm_lambda=0.7))
trials.append(run_trial('jm = 1.0', 'jm', jm_lambda=1.0))

df = pd.DataFrame(trials)
print(df)

In [ ]:
%matplotlib inline

import matplotlib
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')

indexes = 10 * df.index

fig = plt.figure()
ax = plt.subplot(111)

r1 = ax.bar(indexes, df.error_test, width=4)
r2 = ax.bar(indexes + 4, df.error_validate, width=4, color='y')
ax.legend((r1[0], r2[0]), ('Test', 'Validation'))

plt.xticks(indexes, df.name, rotation=70)
plt.ylabel('Error Rate')
plt.xlabel('Smooting Type')
plt.title('NB Error Rates By Smoothing Type')

In [ ]:
## HW1.5: Remove words with frequency of less than three (3) in the training set
Repeat HW1.4. This time when modeling and classification ignore tokens with a frequency of less than three (3) in the training 
set. How does it affect the misclassifcation error of learnt naive multinomial Bayesian Classifier on the training dataset. 
Report the error and the change in error. HINT: ignore tokens with a frequency of less than three (3). Think of this as a 
preprocessing step. How many new mapreduce jobs do you need to solve thus homework?

In [ ]:
import pandas as pd

trials = []

trials.append(run_trial('unsmoothed', 'none', min_word_frequency=3))
trials.append(run_trial('laplace', 'laplace', min_word_frequency=3))
trials.append(run_trial('jm = 0.0', 'jm', jm_lambda=0.0, min_word_frequency=3))
trials.append(run_trial('jm = 0.1', 'jm', jm_lambda=0.1, min_word_frequency=3))
trials.append(run_trial('jm = 0.3', 'jm', jm_lambda=0.3, min_word_frequency=3))
trials.append(run_trial('jm = 0.5', 'jm', jm_lambda=0.5, min_word_frequency=3))
trials.append(run_trial('jm = 0.7', 'jm', jm_lambda=0.7, min_word_frequency=3))
trials.append(run_trial('jm = 1.0', 'jm', jm_lambda=1.0, min_word_frequency=3))

df = pd.DataFrame(trials)
print(df)

## HW1.6 Benchmark your code with the Python SciKit-Learn implementation of the multinomial Naive Bayes algorithm

## HW1.6.0: Multinomial Naive Bayes using SciKit-Learn

It always a good idea to benchmark your solutions against publicly available libraries such as SciKit-Learn, The Machine 
Learning toolkit available in Python. In this exercise, we benchmark ourselves against the SciKit-Learn implementation of 
multinomial Naive Bayes. For more information on this implementation see: 
http://scikit-learn.org/stable/modules/naive_bayes.html more In this exercise, please complete the following:
    
-Run the Multinomial Naive Bayes algorithm (using default settings) from SciKit-Learn over the same training data used in 
HW1.4.2 and report the misclassification error (please note some data preparation might be needed to get the Multinomial Naive 
Bayes algorithm from SkiKit-Learn to run over this dataset)

-Prepare a table to present your results, where rows correspond to approach used (SkiKit-Learn versus your Hadoop 
implementation) and the column presents the misclassification error rates (train, validation, testing) 

-Explain/justify any differences in terms 
of training error rates over the dataset in HW1.5 between your Multinomial Naive Bayes implementation (in Map Reduce) versus 
the Multinomial Naive Bayes implementation in SciKit-Learn

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import *

import csv
import numpy as np

# read email message, and organize training data
with open('enronemail_1h.txt', 'r') as f:
    reader = csv.reader(f, delimiter="\t")
    emails = list(reader)
train_label = [msg[1] for msg in emails]
train_data = [msg[2] + msg[3] if len(msg) == 4 else msg[2] for msg in emails]
msg_id = [msg[0].lower() for msg in emails]
# print(train_label, train_data, msg_id)

# feature vectorization
uniVectorizer = CountVectorizer()
dtmTrain = uniVectorizer.fit_transform(train_data) 
# print(uniVectorizer, dtmTrain)

# multinomial Naive Bayes Classifier from sklearn
mnb = MultinomialNB()
mnb.fit(dtmTrain, train_label)
pred_mnb = mnb.predict(dtmTrain)
training_error_mnb = 1.0 * sum(pred_mnb != train_label) / len(train_label)

# Bernoulli Naive Bayes Classifier from sklearn
bnb = BernoulliNB()
bnb.fit(dtmTrain, train_label)
pred_bnb = bnb.predict(dtmTrain)
training_error_bnb = 1.0*sum(pred_bnb != train_label) / len(train_label)

print 'SK- multinomial NB training error: %.4f' %training_error_mnb
print 'SK- Bernoulli   NB training error: %.4f' %training_error_bnb